In [33]:
import fasttext

"""
The training file should be this format:

__label__pp this is my text
__label__psoe this is also my text
"""

'\nThe training file should be this format:\n\n__label__pp this is my text\n__label__psoe this is also my text\n'

In [ ]:
import fasttext.util
fasttext.util.download_model('es', if_exists='ignore')  # English
ft = fasttext.load_model('cc.es.300.bin')

In [34]:
import pandas as pd
import numpy as np
import re
import emoji

#Function to get all emojis in a text.
def split_emojis(text):

    emoji_list = []
    data = re.findall(r'\X', text)
    for word in data:
        if any(char in emoji.UNICODE_EMOJI for char in word):
            emoji_list.append(word)

    return emoji_list

#Function to clean the text of a tweet.
def clean_tweet(text):
    clean_text = re.sub("@.*:", "", text) #This do not improve performance.
    clean_text = re.sub("@.* ", "", clean_text)
    clean_text = re.sub("RT", "", clean_text)
    clean_text = re.sub("\n", " ", clean_text)
    #Separate emojis from words.
    """
    sep_emojis_text = ""
    for c in clean_text:
        if c in emoji.UNICODE_EMOJI:
            sep_emojis_text+=' '+c+' '
        else:
            sep_emojis_text+=c
     #This do not improve performance.
    """
    return clean_text

#Read and clean data for each party.

#Podemos.
up_cleaned_data = pd.DataFrame()
up_df = pd.read_csv ('up.csv').head(4950)
up_cleaned_data["tweet"] = up_df["text"].apply(clean_tweet)
up_cleaned_data["label"] = "__label__up"
print(len(up_df))
#Psoe.
psoe_cleaned_data = pd.DataFrame()
psoe_df = pd.read_csv ('psoe.csv').head(4950)
psoe_cleaned_data["tweet"] = psoe_df["text"].apply(clean_tweet)
psoe_cleaned_data["label"] = "__label__psoe"
print(len(psoe_df))
#CS.
cs_cleaned_data = pd.DataFrame()
cs_df = pd.read_csv ('cs.csv').head(4950)
cs_cleaned_data["tweet"] = cs_df["text"].apply(clean_tweet)
cs_cleaned_data["label"] = "__label__cs"
print(len(cs_df))
#PP.
pp_cleaned_data = pd.DataFrame()
pp_df = pd.read_csv ('pp.csv').head(4950)
pp_cleaned_data["tweet"] = pp_df["text"].apply(clean_tweet)
pp_cleaned_data["label"] = "__label__pp"
print(len(pp_df))
#Vox.
vox_cleaned_data = pd.DataFrame()
vox_df = pd.read_csv ('vox.csv').head(4950)
vox_cleaned_data["tweet"] = vox_df["text"].apply(clean_tweet)
vox_cleaned_data["label"] = "__label__vox"
print(len(vox_df))

#Concat all parties data.
data = pd.concat([up_cleaned_data, psoe_cleaned_data, cs_cleaned_data, pp_cleaned_data, vox_cleaned_data])

4950
4950
4950
4950
4950


In [35]:
#Split data in training and test sets.
import csv
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data["tweet"], data["label"], test_size=0.15, random_state=42)

training = pd.concat([X_train,y_train], axis=1)
display(training)

test = pd.concat([X_test,y_test], axis=1)
display(test)

#Create training file.
training[training.columns.values[::-1]].to_csv('data.train', index=False, sep=' ', header=False, quoting=csv.QUOTE_NONE, quotechar="", escapechar=" ")
#data.to_csv('data.train', index=False, sep=' ', header=False, quoting=csv.QUOTE_NONE, quotechar="", escapechar=" ")

#Create test file.
test[test.columns.values[::-1]].to_csv('data.test', index=False, sep=' ', header=False, quoting=csv.QUOTE_NONE, quotechar="", escapechar=" ")

,tweet,label
3638,Toda España ha sido testigo. Esto del gran ...,__label__cs
1383,"Estáis en la puñetera ruina, tenéis el frigo...",__label__vox
3950,//t.co/Bya95PYsad,__label__cs
1943,//…,__label__up
1712,Afortunadamente algunos medios se dedican a ...,__label__psoe
...,...,...
1775,//t.co/af04…,__label__vox
440,Manifestaciones contra el gobierno gritando: “...,__label__psoe
860,Esta mañana ha llegado el primer SÍ a los Pr...,__label__up
945,"Los abajo firmantes, 20 años después. https://...",__label__pp


,tweet,label
3857,soporto,__label__psoe
1457,Qué gusto poder hablar sin gritos ni ruido ni ...,__label__up
1743,El Gobierno reconoce el trabajo fundamental ...,__label__up
3729,El centro al que va mi hermano es concertado...,__label__psoe
87,Hola 9.1?,__label__pp
...,...,...
1852,Cs,__label__cs
766,Todo mi cariño a las personas que no pueden ...,__label__pp
4202,‼️ Q raro ‼️ #Bruselas llamando mentiroso a 😆,__label__cs
1125,Todo lo que dice el presidente es correcto y...,__label__pp


In [1]:
import fasttext
# Load input training set.

#By default, fastText sees each training example only five times during training, which is pretty small. The number of times each examples is seen (also known as the number of epochs), can be increased using the -epoch option
model = fasttext.train_supervised(input="data.train", epoch=25, pretrainedVectors = "cc.es.300.bin")

ValueError: cc.es.300.bin cannot be opened for loading!

In [37]:
#To predict one instance.
print( model.predict("Franco deberia volver a vivir") )
print( model.predict("Los españoles son los mejores 💙") )
print( model.predict("Los españoles son los mejores 💚") )
print( model.predict("España es una gran nacion") )
print( model.predict("Pablo Iglesias es el mejor") )
print( model.predict("Pablo Iglesias es un mierdas") )
print( model.predict("Putos progres de mierda") )
print( model.predict("Viva la guardia civil") )
print( model.predict("Igualdad salarial para las mujeres!") )
print( model.predict("La españa moderada") )
print( model.predict("La mujer a trabajar") )
print( model.predict("Viva la fuerza naranja") )

(('__label__vox',), array([0.92883462]))
(('__label__pp',), array([0.99894482]))
(('__label__vox',), array([0.90336657]))
(('__label__cs',), array([0.98046625]))
(('__label__up',), array([0.49434617]))
(('__label__up',), array([0.96629602]))
(('__label__vox',), array([1.00000906]))
(('__label__up',), array([0.71149999]))
(('__label__up',), array([0.89277411]))
(('__label__cs',), array([0.96424067]))
(('__label__cs',), array([0.99996185]))
(('__label__up',), array([0.99918598]))


In [38]:
#To test the test set. (number of samples, precision, recall)
model.test("data.test")

precision = model.test("data.test")[1]
recall = model.test("data.test")[2]

F_score = 2*(precision * recall)/(precision + recall)

print("Precision:", precision)
print("Recall:", recall)
print("Fscore:", F_score)

Precision: 0.4915162941018045
Recall: 0.4915162941018045
Fscore: 0.4915162941018045


In [39]:
model.save_model("fastTextModel.model")